Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [2]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import pickle

First reload the data we generated in 1_notmnist.ipynb.

In [3]:
pickle_file = '../../data/notMNIST_sanitized.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset_sanitized']
    train_labels = save['train_labels_sanitized']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save # hint to help gc free up memory
    print('Training_set',train_dataset.shape, train_labels.shape)
    print('Validation_set',valid_dataset.shape, valid_labels.shape)
    print('Test_set',test_dataset.shape, test_labels.shape)

Training_set (192407, 28, 28) (192407,)
Validation_set (10000, 28, 28) (10000,)
Test_set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

In [5]:
def reformat(dataset, labels):
    """重构数据集样式"""
    dataset = dataset.reshape(dataset.shape[0], -1).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32) 
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training_set',train_dataset.shape, train_labels.shape)
print('Validation_set',valid_dataset.shape, valid_labels.shape)
print('Test_set',test_dataset.shape, test_labels.shape)

Training_set (192407, 784) (192407, 10)
Validation_set (10000, 784) (10000, 10)
Test_set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [6]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():
    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels])) # 权重
    biases = tf.Variable(tf.zeros([num_labels]))# 偏置
    
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

W0906 21:54:01.566587 10500 deprecation.py:323] From <ipython-input-6-d7604aed6641>:29: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Let's run this computation and iterate:

In [7]:
nums_steps = 801

def accuracy(predictions, labels):
#     correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(labels, 1)) # bool
#     print(correct_prediction)
#     return tf.reduce_mean(tf.cast(correct_prediction, "float")) # tf.cast:将bool转换成其他类型
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases.
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(nums_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if step % 100 == 0:
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels)) # eval() return just one tensor
            print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 16.176167
Training accuracy: 14.8%
Validation accuracy: 14.4%
Test accuracy: 15.2%
Loss at step 100: 2.270991
Training accuracy: 72.0%
Validation accuracy: 71.5%
Test accuracy: 78.2%
Loss at step 200: 1.840266
Training accuracy: 74.8%
Validation accuracy: 73.7%
Test accuracy: 80.0%
Loss at step 300: 1.606147
Training accuracy: 76.2%
Validation accuracy: 74.5%
Test accuracy: 81.0%
Loss at step 400: 1.444283
Training accuracy: 77.0%
Validation accuracy: 74.9%
Test accuracy: 81.5%
Loss at step 500: 1.322286
Training accuracy: 77.7%
Validation accuracy: 75.0%
Test accuracy: 81.8%
Loss at step 600: 1.226327
Training accuracy: 78.2%
Validation accuracy: 75.0%
Test accuracy: 82.0%
Loss at step 700: 1.148601
Training accuracy: 78.7%
Validation accuracy: 75.2%
Test accuracy: 82.3%
Loss at step 800: 1.084132
Training accuracy: 79.2%
Validation accuracy: 75.3%
Test accuracy: 82.4%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [8]:
batch_size = 128

gragh = tf.Graph()
with gragh.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Train computation
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [9]:
num_steps = 3001

with tf.Session(graph=gragh) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset:batch_data, tf_train_labels:batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if step % 500 == 0:
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 16.211447
Minibatch accuracy: 16.4%
Validation accuracy: 12.5%
Test accuracy: 13.0%
Minibatch loss at step 500: 1.572111
Minibatch accuracy: 75.0%
Validation accuracy: 75.3%
Test accuracy: 82.4%
Minibatch loss at step 1000: 1.162931
Minibatch accuracy: 80.5%
Validation accuracy: 76.9%
Test accuracy: 83.9%
Minibatch loss at step 1500: 1.313336
Minibatch accuracy: 79.7%
Validation accuracy: 77.1%
Test accuracy: 84.1%
Minibatch loss at step 2000: 1.496679
Minibatch accuracy: 71.9%
Validation accuracy: 77.5%
Test accuracy: 84.6%
Minibatch loss at step 2500: 0.845301
Minibatch accuracy: 81.2%
Validation accuracy: 78.0%
Test accuracy: 85.3%
Minibatch loss at step 3000: 1.255999
Minibatch accuracy: 75.8%
Validation accuracy: 77.8%
Test accuracy: 84.9%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [10]:
# 1-hidden layer neural network
import math
batch_size = 10000
hidden1_units = 1024

with tf.name_scope('hidden1') as scope:
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variable
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden1_units],
                           stddev=1.0 / math.sqrt(float(image_size * image_size))))
    biases1 = tf.Variable(tf.zeros([hidden1_units]))
    
    # hidden layer 1
    hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    
    weights2 = tf.Variable(
        tf.truncated_normal([hidden1_units, num_labels],
                           stddev=1.0 / math.sqrt(float(hidden1_units))))
    biases2 = tf.Variable(tf.zeros([num_labels]))

    # Train computation
    logits = tf.matmul(hidden1, weights2) + biases2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
     
    # Optimizer
#     tf.scalar_summary(loss.op.name, loss)
    global_step = tf.Variable(0, name='global_step', trainable=False) # 保存全局训练步骤（global training step）的数值
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss, global_step=global_step)
    
    # Predictions for the training
    train_prediction = tf.nn.softmax(logits)
    
    # validation
    hidden_valid = tf.nn.relu(tf.matmul(valid_dataset, weights1) + biases1)
    valid_prediction = tf.nn.softmax(tf.matmul(hidden_valid, weights2) + biases2)
    
    # test
    hidden_test = tf.nn.relu(tf.matmul(test_dataset, weights1) + biases1)
    test_prediction = tf.nn.softmax(tf.matmul(hidden_test, weights2) + biases2)

In [12]:
num_steps = 500

with tf.Session() as session:
    
    tf.global_variables_initializer().run() # 初始化变量
    print("Initialized")
    for step in range(num_steps):
        # dataset and labesl
        batch_data = train_dataset[:batch_size, :]
        batch_labels = train_labels[:batch_size, :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        feed_dict = {tf_train_dataset:batch_data, tf_train_labels:batch_labels}
        # Run the computations. 
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if step % 50 == 0:
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.318545
Minibatch accuracy: 9.1%
Validation accuracy: 57.2%
Test accuracy: 62.6%
Minibatch loss at step 50: 0.535350
Minibatch accuracy: 84.8%
Validation accuracy: 82.7%
Test accuracy: 89.7%
Minibatch loss at step 100: 0.448310
Minibatch accuracy: 87.3%
Validation accuracy: 83.6%
Test accuracy: 90.5%
Minibatch loss at step 150: 0.383225
Minibatch accuracy: 89.3%
Validation accuracy: 83.8%
Test accuracy: 90.7%
Minibatch loss at step 200: 0.327056
Minibatch accuracy: 91.2%
Validation accuracy: 84.1%
Test accuracy: 90.9%
Minibatch loss at step 250: 0.280076
Minibatch accuracy: 92.8%
Validation accuracy: 84.1%
Test accuracy: 90.9%
Minibatch loss at step 300: 0.236709
Minibatch accuracy: 94.1%
Validation accuracy: 84.1%
Test accuracy: 90.8%
Minibatch loss at step 350: 0.198386
Minibatch accuracy: 95.4%
Validation accuracy: 84.3%
Test accuracy: 91.1%
Minibatch loss at step 400: 0.166405
Minibatch accuracy: 96.7%
Validation accuracy: 84.2%
Test accuracy: